In [1]:
import math
import struct
from encodings.johab import codec
from http.cookiejar import offset_from_tz_string
from os import name


# First Generation Lunar Use
class SPDU4:
    # Class attributes here: shared constants
    DIRECTIVE_NAMES_MAP = {
        0b000: "Link Establishment & Control",
        0b001: "Report Request",
        0b010: "Set V(R)",
        0b011: "Report Source Spacecraft ID",
        0b100: "PN Ranging",
        # 0b100 to 0b111: Reserved
    }





    """ Set V(R) """
    SET_VR_FIELDS = [
        ("directive_name", 0, 3),
        ("spare", 3, 5),
        ("seq_ctrl_fsn", 8, 8),
    ]

    """ Report Source Spacecraft ID"""
    REPORT_SOURCE_SCID_FIELDS = [
        ("directive_name", 0, 3),
        ("reserved", 3, 13),
        ("source_SCID", 16, 16),
    ]

    """ PN Ranging """
    PN_RANGING_FIELDS = [
        ("directive_name", 0, 3),
        ("mode_type", 3, 2),
        ("ranging_code", 5, 2),
        ("chip_rate_k", 7, 3),
        ("chip_rate_l", 10, 14),
        ("chip_rate_m", 24, 14),
        ("ranging_mod_index", 38, 3),
        ("pn_epoch_time_tag_day", 41, 16),
        ("pn_epoch_time_tag_ms_of_day", 57, 32),
        ("status_report_request", 89, 5),
        ("spares", 94, 2),
    ]

    def __init__(self):
        self.directive_name = None

    def handle_directive(self, directive_code: int, payload: bytes = b''):
        """Dispatch to directive logic"""
        self.directive_name = DIRECTIVE_NAMES_MAP.get(directive_code, "Reserved")
        match directive_code:
            case 0b0000: # #Link Establishment & Control
                self.lec_directive = LECDirective.from_payload(payload)
                self.lec_directive.process()
            case 0b001:
                self.rr_directive = RRDirective.from_payload(payload)
                self.rr_directive.process()
                self.
            case 0b010:
                self._set_vr(payload)
            case 0b011:
                self._report_source_scid(payload)
            case 0b100:
                self._pn_ranging(payload)
            case _:
                print(f"Reserved directive: {directive_code:03b}")

    class LECDirective:
        """Link Establishment and Control Directive"""
        LEC_FIELDS = [ # (name, bit_start, bit_length)
            ("directive_name", 0, 3),
            ("link_direction", 3, 1),
            ("demand_query", 4, 1),
            ("query_response", 5, 1),
            ("remote_no_more_data", 6, 1),
            ("token", 7, 1),
            ("duplex_simplex", 8, 3),
            ("frequency", 11, 5),
            ("polarization", 16, 1),
            ("modulation_index", 17, 3),
            ("modulation", 20, 4),
            ("spares", 24, 2),
            ("coding", 26, 6),
            ("link_snr", 32, 8),
            ("symbol_rate", 40, 16),
        ]

        LINK_DIRECTION_MAP = {
            0b0: "Return Link",
            0b1: "Forward Link",
        }

        DEMAND_QUERY_MAP = {
            0b0: "Demand (Command)",
            0b1: "Query (Link Negotiation)",
        }

        QUERY_RESPONSE_MAP = {
            0b0: "ACK",
            0b1: "NACK",
        }
        RNMD_MAP = {
            0b0: "No Change",
            0b1: "Remote Node has No More Data to Send (RNMD)",
        }

        TOKEN_MAP = {
            0b0: "No Change",
            0b1: "Transmit",
        }

        DUPLEX_SIMPLEX_MAP = {
            0b000: "No Change",
            0b001: "Full Duplex",
            0b010: "Half Duplex",
            0b011: "Simplex Transmit",
            0b100: "Simplex Receive",
            # 0b101-0b111: Reserved
        }

        # Forward: 5-bit binary code -> channel (0-31)
        FORWARD_FREQUENCY_MAP = {i: f"Forward Channel {i}" for i in range(32)}
        # Return: 5-bit binary code -> channel (0-31)
        RETURN_FREQUENCY_MAP = {i = f"Return Channel {i}" for i in range(32)}

        POLARIZATION_MAP = {
            0b0: 'Left Hand CP',
            0b1: 'Right Hand CP'
        }

        MODULATION_INDEX_MAP = {
            0b000: 0.0,
            0b001: 0.4,
            0b010: 0.6,
            0b011: 0.8,
            0b100: math.pi/3,
            0b101: 1.15,
            0b110: 1.3,
            0b111: 1.4,
        }

        MODULATION_MAP = {
            0b0000: "PCM/PM/Bi-phase-L (filtered)",
            0b0001: "GMSK",
            # Others reserved
        }

        CODING_MAP = {
            0b000000: "Uncoded",
            0b000001: "LDPC(2048, 1024)",
            0b000101: "LDPC(6144, 4096)",
            0b001010: "LDPC(8160, 7136)",
            # Others reserved
        }

        def __init__(self, data: bytes):
            if len(data) != 7:
                raise ValueError("LEC 7 bytes")
            self.bits = int.from_bytes(data, 'big')
            self.fields = {name: start for name, start, _ in LEC_FIELDS}

        def _extract(self, name:str, length: int = None) -> int:
            """Bits from LEC_FIELDS"""
            start = self.fields[name]
            length = next(1 for n, s, 1 in LEC_FIELDS if n == name)
            return (self.bits >> start) & ((1 << length)-1)

        @classmethod
        def from_payload(cls, payload: bytes) -> 'LECDirective':
            if len(payload) < 7:
                raise ValueError("Insufficient payload for LEC")
            bits = int.from_bytes(payload[:7], 'big')
            return cls(bits)

        def process(self):
            pass
        def demand_query(self) -> str:
            val = self._extract("demand_query")
            return DEMAND_QUERY_MAP.get(val, "Invalid")
        def query_response(self) -> str:
            val = self._extract("query_response")
            return QUERY_RESPONSE_MAP.get(val, "Invalid")
        def rnmd(self) -> str:
            val = self._extract("rnmd")
            return RNMD_MAP.get(val, "Invalid")
        def token(self) -> str:
            val = self._extract("token")
            return TOKEN_MAP.get(val, "Invalid")
        def duplex_simplex(self) -> str:
            """ Mode Selector """
            val = self._extract("duplex_simplex")
            return DUPLEX_SIMPLEX_MAP.get(val, "Reserved")
        def frequency(self):
            pass
        def polarization(self) -> str:
            val = self._extract("polarization")
            return POLARIZATION_MAP.get(val, "Invalid")
        def mod_index(self) -> float:
            val = self._extract(self, "mod_index")
            return MODULATION_INDEX_MAP.get(val, "Invalid")
        def modulation(self):
            pass
        def spares(self):
            pass
        def coding(self):
            val = self._extract("coding")
            return CODING_MAP.get(val, "RESERVED")
        def instantaneous_link_snr(self):
            pass
        def symbol_rate(self):
            """
            Encode symbol rate (sym/s) -> 16-bit IEEE 754 half-precision

            Range: 1/256 to 2^31 sym/s, precision 0.1%
            Sign bit always 0 (positive)
            :return: Symbol rate (sym/s)
            :rtype: float
            """

            # Pack/Unpack 16-bit symbol rate field
            # Scale: sym/s * 2^-16 -> IEEE range
            scaled = symbol * (1/ 65535.0)

            # Pack as half-precision
            half_bytes = struct.pack('>e', scaled)
            bits = int.from_bytes(half_bytes, 'big')

            # Validation - CCSDS Spec Rules
            sign_bit = (bits >> 15) & 1
            exponent = (bits >> 10) & 0x1F

            if sign_bit != 0:
                raise ValueError

    class RR_Directive(self, payload: bytes):
            """ Report Request """
    RR_FIELDS = [ # (name, bit_start, bit_length)
        ("directive_name", 0, 3),
        ("pcid0_plcw_request", 3, 1),
        ("pcid1_plcw_request", 4, 1),
        ("time_tag_sample_request", 5, 6),
        ("status_report_request", 11, 5),
    ]
        def __init__(self):
             if len(data) != 2:
                raise ValueError("Report Request 2 bytes")
            self.bits = int.from_bytes(data, 'big')
            self.fields = {name: start for name, start, _ in RR_FIELDS}

       @classmethod
        def from_payload(cls, payload: bytes) -> 'LECDirective':
            if len(payload) < 7:
                raise ValueError("Insufficient payload for LEC")
            bits = int.from_bytes(payload[:7], 'big')
            return cls(bits)
        def directive_name(self):
            pass

        def pcid0_plcw_request(self):
            pass

        def pcid_1_plcw_request(self):
            pass

        def time_tag_sample_request(self):
            pass

        def status_report_request(self):
            pass
    def _set_vr(self, payload: bytes):
        """Update V(R) from bits."""
        directive name
        receiver frame sequence number seq ctrl fsn
    def _report_scid(self, payload: bytes):
        """Extract SCID from bits"""
        directive name
        reserved
        source spacecraft ID
    def _pn_ranging(self, payload: bytes):
        directive name
        mode type
            ranging off
            one-way ranging
            two-way non-regenerative ranging
            two-way regenerative ranging
        ranging code

        chip rate (k,l,m)
        ranging mod index
        pn epoch time tag
        status report request
        spares

SyntaxError: unmatched '}' (2870039683.py, line 41)